<h1><center><u>Trabajo Practico Estructuras de datos</u></center></h1>

<h2><u>Primera parte:</u></h2>

<h4><font  size="3" face="Times new romans">•El trabajo practico consiste en la recolección masiva de Tweets sobre un tema de actualidad. Nuestra selección fue todo lo referido a la pandemia, covid-19 y la vacunación. Esto lo empleamos en nuestra QUERY ampliada, sobre el live Stream visto en la clase 7 "Redes sociales", el cual modificamos de tal forma que cada vez que encuentra un tweet según la query, tome los datos del usuario, fecha del tweet, su id y el texto almacenándolos en un archivo .json llamado corpus.</font></h4>

In [5]:

import sys
import time
if __name__ == '__main__':

    from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, HydrateType, OAuthType
    import json

    def stream_tweets(query, expansions, tweet_fields, user_fields):
        
        try:
            o = TwitterOAuth.read_file()
            api = TwitterAPI(o.consumer_key, o.consumer_secret, auth_type=OAuthType.OAUTH2, api_version='2')

            r = api.request('tweets/search/stream/rules', {'add': [{'value':QUERY}]})
            json.dumps(r.json(), indent=2)
            if r.status_code != 201: exit()

            r = api.request('tweets/search/stream/rules', method_override='GET')
            json.dumps(r.json(), indent=2)
            if r.status_code != 200: exit()

            r = api.request('tweets/search/stream', {
                    'expansions': expansions,
                    'tweet.fields': tweet_fields,
                    'user.fields': user_fields,
                },
                hydrate_type=HydrateType.APPEND)

            print(f'[{r.status_code}] START...')
            print("La busqueda empezo a la/s --> " + time.strftime('%H:%M:%S', time.localtime()))
            
            contador_de_tweets = 0
            contador_de_tamanio = 0
            info = {}
            for item in r:
                for c,v in item.items():
                    try:
                        info[v['conversation_id']] = {'texto' : v['text'], 'nombre_oficial' : v['author_id_hydrate']['name'], 'cuenta' : v['author_id_hydrate']['username'], 'id_usuario' : v['author_id'], 'fecha' : v['created_at']}
                        if 'location' in v['author_id_hydrate']:
                            info[v['conversation_id']]['ubicacion'] = v['author_id_hydrate']['location']
                        if 'source' in v:
                            info[v['conversation_id']]['fuente'] = v['source']
                        contador_de_tweets += 1
                        print("Tweets encontrados hasta el momento --> " + str(contador_de_tweets))
                        contador_de_tamanio += sys.getsizeof(info[v['conversation_id']])
                        print("Tamano del tweet hasta el momento -->  " + str(contador_de_tamanio) + ' bytes\n')

                        if contador_de_tweets == 1000:
                            contador_de_tweets = 0
                            with open('corpus.json',"r", encoding="utf-8") as file_corpus:
                                datos_previos = json.load(file_corpus)
                                datos_previos.update(info)
                                print('-------------------------------------------')
                                print("BACKUP: Cantidad total de tweets almacenados: "+str(len(datos_previos)))
                                print('-------------------------------------------'+ '\n')

                            with open('corpus.json',"w", encoding="utf-8") as file_corpus:
                                jsonobj = json.dumps(datos_previos, indent=2)
                                file_corpus.write(jsonobj)
                        
                    except Exception as e:
                        print(e)
                    
        except KeyboardInterrupt:
            print('\nDone!')
            print("La busqueda termino a la/s --> " + time.strftime('%H:%M:%S', time.localtime()))
            print("Tweets encontrados --> " + str(contador_de_tweets))
            print("Tamanio total de los tweets --> " + str(contador_de_tamanio)+ ' bytes')
            try:
                with open('corpus.json',"r", encoding="utf-8") as file_corpus:
                    datos_previos = json.load(file_corpus)
                    datos_previos.update(info)
                    print("Cantidad total de tweets almacenados: "+str(len(datos_previos)))

                with open('corpus.json',"w", encoding="utf-8") as file_corpus:
                    jsonobj = json.dumps(datos_previos, indent=2)
                    file_corpus.write(jsonobj)
            except FileNotFoundError as e:
                with open('corpus.json',"w", encoding="utf-8") as file_corpus:
                    jsonobj = json.dumps(info, indent=2)
                    file_corpus.write(jsonobj)

        except TwitterRequestError as e:
            print(f'\n{e.status_code}')
            for msg in iter(e):
                print(msg)

        except TwitterConnectionError as e:
            print(e)

        except Exception as e:
            print(e)

    QUERY = '(vacuna OR covid OR pandemia OR #covid-19 OR #covid19 OR Sputnik OR Astrazeneca OR Sinopharm) (-muerte) lang:es (-is:retweet) (-is:reply)'
    EXPANSIONS = 'author_id,referenced_tweets.id,referenced_tweets.id.author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids,geo.place_id,entities.mentions.username'
    TWEET_FIELDS='author_id,conversation_id,created_at,entities,geo,id,lang,public_metrics,source,text'
    USER_FIELDS='created_at,description,entities,location,name,profile_image_url,public_metrics,url,username'
    stream_tweets(QUERY, EXPANSIONS, TWEET_FIELDS, USER_FIELDS)


[200] START...
La busqueda empezo a la/s --> 21:17:57

Done!
La busqueda termino a la/s --> 21:18:01
Tweets encontrados --> 0
Tamanio total de los tweets --> 0 bytes
Cantidad total de tweets almacenados: 121


<h4><font  size="3" face="Times new romans">•Tomando el código visto en clase sobre BSBI y optimizarlo para reducir sus coste de ejecución, pudimos construir nuestro indice invertido. A partir de nuestros corpus.json realizamos nuestros diccionarios de documentos(ubicación del corpus que contiene toda la información de cada tweet) y nuestros postings.json(que guardan las id de cada termino apuntando los documentos donde aparecen y la id de cada tweet).</font></h4>

In [6]:
import json
import os
import re

class Indice_BSBI(object):
    def __init__(self, salida, temporal, documentos, blocksize):
        self.documentos = documentos
        self.salida = salida
        self._blocksize = blocksize
        self._temp = temporal
        
    def generar_docID(self):
            doc_a_docID = {}
            docID_to_doc = {}
            docs_list = re.findall(r'(?<=Practico\/)\S*', self.documentos)

            for i in range(len(docs_list)):
                doc_a_docID[docs_list[i]] = i
            self.docs_list = docs_list
            self._doc_a_docID = doc_a_docID

    def invertir_bloque(self, bloque):
        bloque_invertido = {}
        bloque_ordenado = sorted(bloque, key=lambda tuple: (tuple[0], tuple[1]))
        for par in bloque_ordenado:
            posting = bloque_invertido.setdefault(par[0], set())
            posting.add(par[1])
        return bloque_invertido

    def guardar_bloque_invertido(self, bloque, numero_bloque):
        archivo_de_salida = "b"+str(numero_bloque)+".json"
        archivo_de_salida = os.path.join(self._temp, archivo_de_salida)
        for clave in bloque:
            bloque[clave] = list(bloque[clave])
        with open(archivo_de_salida, "w") as contenedor:
            json.dump(bloque, contenedor)
        return archivo_de_salida

    def guardar_dict_docs(self):
        self._doc_a_docID = { doc_id : doc for doc, doc_id in self._doc_a_docID.items()}
        path = os.path.join(self.salida, "dict_docs.json")
        with open(path, "w") as contenedor:
            json.dump(self._doc_a_docID, contenedor)
    
    def guardar_index_dict(self, dict_path, dict):
        path = os.path.join(self.salida, dict_path)
        with open(path, "w") as container:
            json.dump(dict, container)

    def intercalar_bloques(self, archivos_temp, archivos_ID):
        lista_term_ID=[str(i) for i in range(len(archivos_ID))]
        posting = os.path.join(self.salida,"postings.json")
        lista_de_bloques=[]
        dict_postings = {}

        open_files = [open(f, "r") for f in archivos_temp]
        for data in open_files:
            data.seek(0)
            lista_de_bloques.append(json.load(data))
        
        with open(posting,"w") as output:
            for termID in lista_term_ID:
                dict_docID_tweetID = {}
                for bloque in lista_de_bloques:
                    try:
                        for lista in bloque[termID]:
                            if lista[0] not in dict_docID_tweetID.keys():
                                dict_docID_tweetID[lista[0]] = []
                            dict_docID_tweetID[lista[0]].append(lista[1])
                    except:
                        pass
                dict_postings.update({termID : dict_docID_tweetID})
            output.write(json.dumps(dict_postings, indent=2))

<h4><font  size="3" face="Times new romans">•Importamos nuestro BSBI para crear dos clases hijas, una que nos construye un diccionario de terminos:id, un diccionario de documentos:id y un diccionario 'postings' que tiene como clave la id de los terminos y como valor un diccionario con id_doc:id_tweet.</font></h4>

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
import re
import time
import json
import os
from BSBI import Indice_BSBI

class Indice_de_palabras(Indice_BSBI):
    def __init__(self, salida, temporal, documentos, blocksize, language='spanish'):
        super().__init__(salida, temporal, documentos,blocksize)
        self._stop_words = frozenset(stopwords.words(language))
        self._stemmer = SnowballStemmer(language, ignore_stopwords=False)
        self._term_to_termID = {}

        self.generar_docID()
        self.__indexar()


    def __lemmatizar(self, palabra):
        palabra = palabra.strip(string.punctuation + "|" + "'" + "´" + "-" + "»" + "\x97" + "¿" + "¡" +\
                                "\u201c" + "\u25b6" + "\u201d" + "\u2014" + "\u2018" + "\u2019" + "\u00bf")

        lemmatized_word = self._stemmer.stem(palabra)
        return lemmatized_word

    def __indexar(self):
        n = 0
        lista_de_bloques = []

        for bloque in self.__parse_next_block():
            bloque_invertido = self.invertir_bloque(bloque)
            lista_de_bloques.append(self.guardar_bloque_invertido(bloque_invertido, n))
            n += 1
        start = time.process_time()
        self.intercalar_bloques(lista_de_bloques, self._term_to_termID)
        end = time.process_time()
        print("Intercalar bloques de palabras ==> Tiempo transcurrido: ", end-start)

        self.guardar_dict_terms()
        self.guardar_dict_docs()

    def __parse_next_block(self):
        blocksize = self._blocksize
        termID = 0
        bloque = []
        for doc in self.docs_list:
            try:
                with open(doc, "r", encoding="utf-8") as corpus:
                    dict_tweet = json.load(corpus)
                    for id, data in dict_tweet.items():

                        blocksize -= len(data['texto'].encode('utf-8'))
                        tweet = self.__sacar_links_y_emojis(data['texto'])
                        for palabra in tweet:
                            
                            if palabra not in self._stop_words and len(palabra) > 1:
                                palabra = self.__lemmatizar(palabra)
                                if palabra not in self._term_to_termID:
                                    self._term_to_termID[palabra] = termID
                                    termID += 1
                                bloque.append((self._term_to_termID[palabra],(self._doc_a_docID[doc], id )))

                        if blocksize <= 0:
                            yield bloque
                            blocksize = self._blocksize
                            bloque = []
                    yield bloque
            except Exception as e:
                print(e)

    def __sacar_links_y_emojis(self, tweet):
        regex_link = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        regex_emoji = '[\u2620-\U0001fa7a]|[\U000e0067-\U000e007f]|[\U000e0062\U000e0065]\
                        |[\U0001fa7a-\U0001fac0]+|[\u200d]+|[\U0001f4bc]+'
        regex_mention = '@[a-zA-Z0-9-]*|#[a-zA-Z0-9À-ÿ\u00f1\u00d1$-_@.&+]*'

        links = set(re.findall(regex_link, tweet))
        emojis = set(re.findall(regex_emoji, tweet))
        menciones = set(re.findall(regex_mention, tweet))
        set_de_caracteres = links.union(emojis.union(menciones))

        for caracter in set_de_caracteres:
            tweet = tweet.replace(caracter, ' ')

        return(tweet.split())

    def guardar_dict_terms(self):
        super().guardar_index_dict("dict_terms.json", self._term_to_termID)

<h4><font  size="3" face="Times new romans">•La otra clase hija construye un diccionario de fecha:id, un diccionario de documentos:id y un diccionario 'postings' que tiene como clave el id de las fechas y como valor un diccionario con id_doc: id_tweet.</font></h4>

In [ ]:
import json
import os
import time
from BSBI import Indice_BSBI

class Indice_de_fechas(Indice_BSBI):
    def __init__(self, salida, temporal, documentos, blocksize):
        super().__init__(salida, temporal, documentos, blocksize)
        self._fecha_a_fechaID = {}

        self.generar_docID()
        self.__indexar()

    def __indexar(self):
        n = 0
        lista_de_bloques = []

        for bloque in self.__parse_next_block():
            bloque_invertido = self.invertir_bloque(bloque)
            lista_de_bloques.append(self.guardar_bloque_invertido(bloque_invertido, n))
            n += 1
        start = time.process_time()
        self.intercalar_bloques(lista_de_bloques, self._fecha_a_fechaID)
        end = time.process_time()
        print("Intercalar bloques de fechas ==> Tiempo transcurrido: ", end-start)

        self.guardar_dict_fechas()
        self.guardar_dict_docs()
    
    def guardar_dict_fechas(self):
        super().guardar_index_dict("dict_dates.json", self._fecha_a_fechaID)

    def __parse_next_block(self):
        blocksize = self._blocksize
        dateID = 0
        bloque = []
        for doc in self.docs_list:
            try:
                with open(doc, "r", encoding="utf-8") as file_corpus:
                    dict_tweet = json.load(file_corpus)
                    for id, data in dict_tweet.items():
                        blocksize -= len(data['texto'].encode('utf-8'))
                        date = self.__limpiar_fechas(data['fecha'])
                        if date not in self._fecha_a_fechaID:
                            self._fecha_a_fechaID[date] = dateID
                            dateID += 1
                        bloque.append((self._fecha_a_fechaID[date],(self._doc_a_docID[doc], id)))

                        if blocksize <= 0:
                            yield bloque
                            blocksize = self._blocksize
                            bloque = []
                    yield bloque
            except Exception as e:
                print(e)

    def __limpiar_fechas(self, tweet):
        fecha = tweet[:4] +'-'+ tweet[5:7] +'-'+ tweet[8:10] +' '+ tweet[11:13] +':'+ tweet[14:16]
        return fecha



<h2><u>Segunda parte:</u></h2>

<font  size="3" face="Times new romans">•Para el menu de busqueda de tweets decidimos utilizar la interfaz de tkinter, creando diferentes ventanas la principal que nos brinda las opciones de ingresar los documentas con los cuales formamos cada uno de los indices.</font>

![titulo](VentanaPrincipal.png )



<font  size="3" face="Times new romans">•La ventana de búsqueda por palabras, cuenta con la opción de ingresar diferentes operadores tanto intersección de palabras, diferencia de palabras y conjunción de palabras. Para realizar esto le matiza las palabras ingresadas, con las cuales implementa una lista de diccionarios con la clave del id del documento y de valor un conjunto con todos los id_tweets del termino en ese documento.</font>


![titulo](VentanaBusquedaPalabra.png )


<font  size="3" face="Times new romans">•La ventana de busqueda por fechas, permite la busqueda entre dos fechas que pueden ser ingresadas en el formato AAA-MM-DD. Las fechas ingresan en un formato string, luego de ser spliteadas se las une en un formato integer para poder compararlas entre si. Luego se recorre el diccionario de fechas:id con las fechas desde y hasta obtiene todos los valores entre ese rango, con estas id se dirije al posting para obtener el todos los id_tweets y de los documentos donde se encuentra para finalizar, en el corpus devolviendo los tweets entre el rango pedido.</font>

![titulo](VentanaBusquedaFecha.png )



<font  size="3" face="Times new romans">•El menu de operaciones es con el cual nosotros podemos realizar la busqueda de las palabras ingresadas y devolver una cantidad pedida de los tweets con estas palabras.</font>

In [ ]:
import json
import re
import string
import tkinter as tk
from tkinter import Listbox, Scrollbar, Tk, ttk
from tkinter import filedialog as fd
from tkinter.constants import BOTH, END, LEFT, RADIOBUTTON, RIGHT, X, Y
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

class Menu_de_operaciones:
    def __init__(self):
        self._stop_words = frozenset(stopwords.words('spanish'))
        self._stemmer = SnowballStemmer('spanish', ignore_stopwords=False)

    def aceptar_fechas(self,d,h,cantidad):
        with open ("output_dates/dict_dates.json","r",encoding="UTF-8") as dict_dates,\
            open ("output_dates/postings.json","r",encoding="UTF-8") as posting,\
            open ("output_dates/dict_docs.json","r",encoding="UTF-8") as dict_docs:
            self.dict_da= json.load(dict_dates)
            self.post= json.load(posting)
            self.dict_doc=json.load(dict_docs)

        desde= self.pasaje_fecha_a_numero(d)
        hasta= self.pasaje_fecha_a_numero(h)

        self.lista_completa = []
        for c,v in self.dict_da.items():
            clave= self.pasaje_fecha_a_numero(c)
            if clave >=desde and clave<=hasta:
                self.obtener_id(str(v))
        self.ventana_nueva_fechas(self.lista_completa,cantidad)

    def obtener_id(self,valor):
        dict_apariciones=self.post[valor]

        for doc_id,list_tweets in dict_apariciones.items():
            with open (str(self.dict_doc[str(doc_id)]),"r",encoding="UTF-8") as corpus:
                self.corpus=json.load(corpus)
                for id in list_tweets:
                    self.lista_completa.append(id)
            
    def pasaje_fecha_a_numero(self,palabras):
        suma=""
        lista = re.split(r'[\W]', palabras)
        for num in lista:
            suma+=num
        return int(suma)
    

    def ventana_nueva_fechas(self, list_tweets, cantidad_de_tweets_pedidos):
            self.ventana1 = Tk()
            self.ventana1.geometry('600x680')
            self.ventana1.resizable(False,False)
            self.ventana1.iconbitmap("Twitter.ico")
            self.labelframe1=ttk.LabelFrame(self.ventana1, text="Búsqueda")        
            self.labelframe1.grid(column=0, row=0, padx=5, pady=10)
            scrollbar = Scrollbar(self.labelframe1)
            scrollbar.pack( side = RIGHT, fill = Y )

            mylist = Listbox(self.labelframe1, yscrollcommand = scrollbar.set, width=95, height=40)
            cantidad_de_tweets = int(cantidad_de_tweets_pedidos)
            if(len(list_tweets)<cantidad_de_tweets-1):
                cantidad_de_tweets = len(list_tweets)-1
            
            for id_tweet in list_tweets[0:int(cantidad_de_tweets_pedidos)]:
                tweet = ("@{} => {}".format(self.corpus[id_tweet]["cuenta"], self.corpus[id_tweet]["texto"]))
                largo = len(tweet.encode('utf-8'))
                mylist.insert(END,'---------------------------------------------------------------------------------------------------------------')
                aux = 0
                for x in range(0,largo//100):
                    mylist.insert(END,tweet[aux:aux+100])
                    aux=aux+100
                mylist.insert(END,tweet[aux:])
            mylist.insert(END,'---------------------------------------------------------------------------------------------------------------')

            mylist.pack( side = LEFT, fill = BOTH )
            scrollbar.config( command = mylist.yview )

            self.ventana1.mainloop()
    
    def aceptar_palabras(self,entrada,negadas,cantidad):
        with open ("output_words/dict_terms.json","r",encoding="UTF-8") as dict_terms,\
            open ("output_words/postings.json","r",encoding="UTF-8") as posting,\
            open ("output_words/dict_docs.json","r",encoding="UTF-8") as dict_docs:
            self.dict_term= json.load(dict_terms)
            self.post= json.load(posting)
            self.dict_doc=json.load(dict_docs)

        lista_dicts_conjuntos = self.conjuntos_id_tweets(entrada)
    
        operacion_final= self.crear_operacion(entrada,negadas)

        self.lista_completa_de_tuplas = []
        contador= 0
        for tweet in operacion_final:
            encontrado = False
            for dict in lista_dicts_conjuntos:
                for c,v in dict.items():
                    if(encontrado == False):
                        for id in v:
                            if (int(id) == tweet) & (contador< int(cantidad)):
                                contador+=1
                                with open(self.dict_doc[c],"r",encoding="UTF-8") as corpus:
                                    self.corpus = json.load(corpus)
                                    self.lista_completa_de_tuplas.append((self.corpus[id]["cuenta"], self.corpus[id]["texto"]))
                                encontrado = True
            
        self.ventana_nueva_palabras(self.lista_completa_de_tuplas)
        
    def ventana_nueva_palabras(self, lista_de_tuplas):
        
        self.ventana2 = Tk()
        self.ventana2.geometry('600x680')
        self.ventana2.resizable(False,False)
        self.ventana2.iconbitmap("Twitter.ico")
        self.labelframe2=ttk.LabelFrame(self.ventana2, text="Búsqueda")        
        self.labelframe2.grid(column=0, row=0, padx=5, pady=10)
        scrollbar = Scrollbar(self.labelframe2)
        scrollbar.pack( side = RIGHT, fill = Y )

        mylist2 = Listbox(self.labelframe2, yscrollcommand = scrollbar.set, width=95, height=40)

        for par in lista_de_tuplas:
            linea = ("@{} => {}".format(par[0], par[1]))
            largo = len(linea.encode('utf-8'))
            mylist2.insert(END,'---------------------------------------------------------------------------------------------------------------')
            aux = 0
            for x in range(0,largo//100):
                mylist2.insert(END,linea[aux:aux+100])
                aux=aux+100
            mylist2.insert(END,linea[aux:])
        mylist2.insert(END,'-------------------------------------------------------------------------------------------------------------------')    
        
        mylist2.pack( side = LEFT, fill = BOTH )
        scrollbar.config( command = mylist2.yview )

        self.ventana2.mainloop()

    def conjuntos_id_tweets(self,entrada):
        lista_lematizadas = [self.lematizar(palabra) for palabra in re.split("[A-Z\s]+", entrada)]
    
        lista_dicts = []
        for palabra in lista_lematizadas:
            dict_doc_sets = {}
            id = self.dict_term[palabra]
            dict_doc_tweet = self.post[str(id)]
            for c,v in dict_doc_tweet.items():
                conjunto = set()
                for id_tweet in v:
                    conjunto.add(id_tweet)
                    dict_doc_sets[c] = conjunto
            lista_dicts.append(dict_doc_sets)
        return lista_dicts

    def conjunto_id_negadas(self,negadas):
        lista_negadas_lematizadas = [self.lematizar(palabra) for palabra in negadas.split(' ')]

        conjunto_id_negadas = set()
        for palabra in lista_negadas_lematizadas:
            id = self.dict_term[palabra]
            for c,v in self.post[str(id)].items():
                for id_tweets in v:
                    conjunto_id_negadas.add(int(id_tweets))
        return conjunto_id_negadas
        
    def crear_operacion(self,entrada,negadas):
        simbolos = ['(',')','A','N','D','O','R',' ']
        operacion_final = ''
        numero_de_palabra = 0
        posicion=0
        entrada+=" "
        lista_dicts_conjuntos = self.conjuntos_id_tweets(entrada)

        while posicion < len(entrada):
            if entrada[posicion] not in simbolos:
                operacion_final+="{"
                for c,v in lista_dicts_conjuntos[numero_de_palabra].items():
                    for id_tweet in v:
                        operacion_final+= id_tweet
                        operacion_final+=","
                operacion_final= operacion_final[:-1]
                operacion_final+="}"
                numero_de_palabra+=1
                booleano_posicion=True
                while booleano_posicion:
                    posicion+=1
                    if(entrada[posicion]) in simbolos:
                        booleano_posicion=False
            else:
                operacion_final+=entrada[posicion]
                posicion+=1

        operacion_final=operacion_final.replace("AND", "&")
        operacion_final=operacion_final.replace("OR", "|")
        operacion_final=eval(operacion_final)
        operacion_final=operacion_final-self.conjunto_id_negadas(negadas)
        return operacion_final    

    def lematizar(self, palabra):
        palabra = palabra.strip(string.punctuation + "|" + "'" + "´" + "-" + "»" + "\x97" + "¿" + "¡" +\
                                "\u201c" + "\u25b6" + "\u201d" + "\u2014" + "\u2018" + "\u2019" + "\u00bf")

        palabra_lematizada = self._stemmer.stem(palabra)
        return palabra_lematizada
